### * 빠른 입력

In [ ]:
import sys
input = lambda: sys.stdin.readline().rstrip()

In [ ]:
import os
import sys
from io import BytesIO, IOBase
 
BUFSIZE = 8192

class FastIO(IOBase):
    newlines = 0
 
    def __init__(self, file):
        self._fd = file.fileno()
        self.buffer = BytesIO()
        self.writable = "x" in file.mode or "r" not in file.mode
        self.write = self.buffer.write if self.writable else None
 
    def read(self):
        while True:
            b = os.read(self._fd, max(os.fstat(self._fd).st_size, BUFSIZE))
            if not b:
                break
            ptr = self.buffer.tell()
            self.buffer.seek(0, 2), self.buffer.write(b), self.buffer.seek(ptr)
        self.newlines = 0
        return self.buffer.read()
 
    def readline(self):
        while self.newlines == 0:
            b = os.read(self._fd, max(os.fstat(self._fd).st_size, BUFSIZE))
            self.newlines = b.count(b"\n") + (not b)
            ptr = self.buffer.tell()
            self.buffer.seek(0, 2), self.buffer.write(b), self.buffer.seek(ptr)
        self.newlines -= 1
        return self.buffer.readline()
 
    def flush(self):
        if self.writable:
            os.write(self._fd, self.buffer.getvalue())
            self.buffer.truncate(0), self.buffer.seek(0)
 
 
class IOWrapper(IOBase):
    def __init__(self, file):
        self.buffer = FastIO(file)
        self.flush = self.buffer.flush
        self.writable = self.buffer.writable
        self.write = lambda s: self.buffer.write(s.encode("ascii"))
        self.read = lambda: self.buffer.read().decode("ascii")
        self.readline = lambda: self.buffer.readline().decode("ascii")

sys.stdin, sys.stdout = IOWrapper(sys.stdin), IOWrapper(sys.stdout)
input = lambda: sys.stdin.readline().rstrip("\r\n")

### * 깊은 재귀시 이용

In [ ]:
sys.setrecursionlimit(100000)

In [ ]:
## PYRIVAL BOOTSTRAP
# https://github.com/cheran-senthil/PyRival/blob/master/pyrival/misc/bootstrap.py
# This decorator allows for recursion without actually doing recursion
## @bootstrap, yield when getting and returning value in recursive functions, end of functions
from types import GeneratorType

def bootstrap(f, stack=[]):
    def wrappedfunc(*args, **kwargs):
        if stack:
            return f(*args, **kwargs)
        else:
            to = f(*args, **kwargs)
            while True:
                if type(to) is GeneratorType:
                    stack.append(to)
                    to = next(to)
                else:
                    stack.pop()
                    if not stack:
                        break
                    to = stack[-1].send(to)
            return to
    return wrappedfunc

### 약수 구하기

In [5]:
def get_divisors(num):
    divisors = set()
    for mod in range(1, int(num ** 0.5) + 1):
        if num % mod == 0:
            divisors.add(mod)
            divisors.add(num // mod)
            
    return sorted(divisors)

get_divisors(12)

[1, 2, 3, 4, 6, 12]

### 에라토스테네스의 체

In [1]:
max_num = 100
ip = [True] * (max_num + 1)
ip[0] = ip[1] = False
for mod in range(2, int(max_num ** 0.5) + 1):
    if ip[mod]:
        for v in range(mod * mod, max_num + 1, mod):
            ip[v] = False
            
primes = [i for i, v in enumerate(ip) if v]

print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


### 오일러의 체

In [1]:
max_num = 100
sp = [0] * (max_num + 1)
primes = []
for i in range(2, max_num + 1):
    if sp[i] == 0:
        sp[i] = i
        primes.append(i)
        
    j = 0
    while j < len(primes) and i * primes[j] <= max_num and primes[j] <= sp[i]:
        sp[i * primes[j]] = primes[j]
        j += 1
        
print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


### 오일러의 체를 이용한 소인수분해

In [6]:
num = 12
r = []
while num > 1:
    r.append(sp[num])
    num //= sp[num]
    
print(r)

[2, 2, 3]


### 기본 소인수분해

In [7]:
def p_facto(num):
    r = []
    for mod in range(2, int(num ** 0.5) + 1):
        while num % mod == 0:
            num //= mod
            r.append(mod)
    
    if num != 1:
        r.append(num)
        
    return r

p_facto(8)

[2, 2, 2]

### nCr

In [1]:
def nCr(n, r):
    if n - r < r:
        r = n - r
        
    num = 1
    for i in range(n, n - r, -1):
        num *= i
    
    denom = 1
    for i in range(2, r + 1):
        denom *= i
        
    return num // denom

r = nCr(n, k)
print(r)

5 2
10


### 모듈러 곱셈 역원 nCk

In [1]:
mod = 1000000007
max_num = 4000000

factos = [0] * (max_num + 1)
factos[0] = 1
invs = [0] * (max_num + 1)
cur = 1
for i in range(1, max_num + 1):
    cur *= i
    cur %= mod
    factos[i] = cur
    
invs[max_num] = pow(factos[max_num], mod - 2, mod)
for i in range(max_num - 1, -1, -1):
    invs[i] = (i + 1) * invs[i + 1] % mod

n, k = map(int, input().split())

r = factos[n] * invs[k] * invs[n - k] % mod
    
print(r)

4 2
6


### 다익스트라

In [ ]:
from heapq import heappush, heappop

inf = float('inf')
dists = [inf] * (n + 1)
dists[start] = 0

heap = []
heappush(heap, (0, start))

while heap:
    dist, value = heappop(heap)
    
    if dist > dists[value]:
        continue
        
    for v, d in li[value]:
        ndist = dist + d
        if ndist < dists[v]:
            dists[v] = ndist
            heappush(heap, (ndist, v))
            
print(dists)

### 플로이드 워셜

In [ ]:
inf = float('inf')

nli = [[inf] * n for _ in range(n)]
for i in range(n):
    nli[i][i] = 0
    
for mid in range(n):
    for i in range(n):
        for j in range(n):
            nli[i][j] = min(nli[i][j], nli[i][mid] + nli[mid][j])
            
for v in nli:
    print(v)

### 삼분 탐색

In [ ]:
def get_value(mid):
    cur = 0
    sv = 0
    for i in range(1, n):
        cur += mid
        sv += abs(li[i] - cur)
        
    return sv

n = int(input())
li = list(map(int, input().split()))

start = 1
end = 1000000000
result = float('inf')
while start <= end:
    mid1 = (start * 2 + end) // 3
    mid2 = (start + end * 2) // 3
    
    t1 = get_value(mid1)
    t2 = get_value(mid2)
    
    result = min(result, t1, t2)
    
    if end - start <= 5:
        for mid in range(start, end + 1):
            result = min(result, get_value(mid))
        break
    
    if t1 >= t2:
        start = mid1
    else:
        end = mid2
        
print(result)

### Sorted multiset

In [1]:
# https://github.com/tatyam-prime/SortedSet/blob/main/SortedMultiset.py
import math
from bisect import bisect_left, bisect_right, insort
from typing import Generic, Iterable, Iterator, TypeVar, Union, List
T = TypeVar('T')

class SortedMultiset(Generic[T]):
    BUCKET_RATIO = 50
    REBUILD_RATIO = 170

    def _build(self, a=None) -> None:
        "Evenly divide `a` into buckets."
        if a is None: a = list(self)
        size = self.size = len(a)
        bucket_size = int(math.ceil(math.sqrt(size / self.BUCKET_RATIO)))
        self.a = [a[size * i // bucket_size : size * (i + 1) // bucket_size] for i in range(bucket_size)]
    
    def __init__(self, a: Iterable[T] = []) -> None:
        "Make a new SortedMultiset from iterable. / O(N) if sorted / O(N log N)"
        a = list(a)
        if not all(a[i] <= a[i + 1] for i in range(len(a) - 1)):
            a = sorted(a)
        self._build(a)

    def __iter__(self) -> Iterator[T]:
        for i in self.a:
            for j in i: yield j

    def __reversed__(self) -> Iterator[T]:
        for i in reversed(self.a):
            for j in reversed(i): yield j
    
    def __len__(self) -> int:
        return self.size
    
    def __repr__(self) -> str:
        return "SortedMultiset" + str(self.a)
    
    def __str__(self) -> str:
        s = str(list(self))
        return "{" + s[1 : len(s) - 1] + "}"

    def _find_bucket(self, x: T) -> List[T]:
        "Find the bucket which should contain x. self must not be empty."
        for a in self.a:
            if x <= a[-1]: return a
        return a

    def __contains__(self, x: T) -> bool:
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        return i != len(a) and a[i] == x

    def count(self, x: T) -> int:
        "Count the number of x."
        return self.index_right(x) - self.index(x)

    def add(self, x: T) -> None:
        "Add an element. / O(√N)"
        if self.size == 0:
            self.a = [[x]]
            self.size = 1
            return
        a = self._find_bucket(x)
        insort(a, x)
        self.size += 1
        if len(a) > len(self.a) * self.REBUILD_RATIO:
            self._build()

    def discard(self, x: T) -> bool:
        "Remove an element and return True if removed. / O(√N)"
        if self.size == 0: return False
        a = self._find_bucket(x)
        i = bisect_left(a, x)
        if i == len(a) or a[i] != x: return False
        a.pop(i)
        self.size -= 1
        if len(a) == 0: self._build()
        return True

    def lt(self, x: T) -> Union[T, None]:
        "Find the largest element < x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] < x:
                return a[bisect_left(a, x) - 1]

    def le(self, x: T) -> Union[T, None]:
        "Find the largest element <= x, or None if it doesn't exist."
        for a in reversed(self.a):
            if a[0] <= x:
                return a[bisect_right(a, x) - 1]

    def gt(self, x: T) -> Union[T, None]:
        "Find the smallest element > x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] > x:
                return a[bisect_right(a, x)]

    def ge(self, x: T) -> Union[T, None]:
        "Find the smallest element >= x, or None if it doesn't exist."
        for a in self.a:
            if a[-1] >= x:
                return a[bisect_left(a, x)]
    
    def __getitem__(self, x: int) -> T:
        "Return the x-th element, or IndexError if it doesn't exist."
        if x < 0: x += self.size
        if x < 0: raise IndexError
        for a in self.a:
            if x < len(a): return a[x]
            x -= len(a)
        raise IndexError

    def index(self, x: T) -> int:
        "Count the number of elements < x."
        ans = 0
        for a in self.a:
            if a[-1] >= x:
                return ans + bisect_left(a, x)
            ans += len(a)
        return ans

    def index_right(self, x: T) -> int:
        "Count the number of elements <= x."
        ans = 0
        for a in self.a:
            if a[-1] > x:
                return ans + bisect_right(a, x)
            ans += len(a)
        return ans
    
ms = SortedMultiset()
ms.add(1)
ms.index(1)
ms.index_right(1)

### 세그먼트 트리

https://codeforces.com/contest/1691/submission/159088060 참고

In [ ]:
class SegmentTree:
    def __init__(self, data, default=10**15, func=lambda a, b: max(a,b)):
        """initialize the segment tree with data"""
        self._default = default
        self._func = func
        self._len = len(data)
        self._size = _size = 1 << (self._len - 1).bit_length()
 
        self.data = [default] * (2 * _size)
        self.data[_size:_size + self._len] = data
        for i in reversed(range(_size)):
            self.data[i] = func(self.data[i + i], self.data[i + i + 1])
 
    def __delitem__(self, idx):
        self[idx] = self._default
 
    def __getitem__(self, idx):
        return self.data[idx + self._size]
 
    def __setitem__(self, idx, value):
        idx += self._size
        self.data[idx] = value
        idx >>= 1
        while idx:
            self.data[idx] = self._func(self.data[2 * idx], self.data[2 * idx + 1])
            idx >>= 1
 
    def __len__(self):
        return self._len
 
    def query(self, start, stop):
        if start == stop:
            return self.__getitem__(start)
        stop += 1
        start += self._size
        stop += self._size
 
        res = self._default
        while start < stop:
            if start & 1:
                res = self._func(res, self.data[start])
                start += 1
            if stop & 1:
                stop -= 1
                res = self._func(res, self.data[stop])
            start >>= 1
            stop >>= 1
        return res
 
    def __repr__(self):
        return "SegmentTree({0})".format(self.data)
    
func = lambda a, b: a * b
segtree = SegmentTree(li, -inf, func=func) # 두 번째 인자는 세그트리 배열의 기본 값
segtree.query(l, r)

### 펜웍 트리

In [ ]:
def get_sum(i):
    total = 0
    while i > 0:
        total += tree[i]
        i -= i & (-i)
        
    return total

def update(i, diff):
    while i < n + 1:
        tree[i] += diff
        i += i & (-i)

n, m = map(int, input().split())

li = [0] * (n + 1)
tree = [0] * (n + 1)

# 합계시
sv = get_sum(b) - get_sum(a - 1)

# 변경시
diff = nv - li[index]
li[index] = nv
update(index, diff)

### LIS (nlogn)

In [ ]:
from bisect import bisect_left

n = int(input())
li = list(map(int, input().split()))

nli = [li[0]]
for i in range(1, len(li)):
    index = bisect_left(nli, li[i])
    
    if index == len(nli):
        nli.append(li[i])
    else:
        nli[index] = li[i]
        
print(len(nli))

### Counting Inversions

In [ ]:
def merge_sort(li):
    global count
    
    if len(li) == 1:
        return li
    
    left = merge_sort(li[:len(li) // 2])
    right = merge_sort(li[len(li) // 2:])
    
    nl = len(left)
    nr = len(right)
    
    nli = []
    l_i = r_i = 0
    while True:
        if left[l_i] <= right[r_i]:
            nli.append(left[l_i])
            count += r_i
            l_i += 1
            
            if l_i == nl:
                while r_i < nr:
                    nli.append(right[r_i])
                    r_i += 1
                break    
        else:
            nli.append(right[r_i])
            r_i += 1
            
            if r_i == nr:
                while l_i < nl:
                    nli.append(left[l_i])
                    count += r_i
                    l_i += 1
                break
    return nli 
    
count = 0
merge_sort(li)

print(count)

### 교란 순열

- 각자가 자신의 것을 제외한 모든 다른 것을 이용할 수 있는 경우

In [ ]:
mod = 1000000000

dp = [0] * 1000001
dp[:4] = [1, 0, 1, 2]
for i in range(4, 1000001):
    dp[i] = (dp[i - 2] + dp[i - 1]) * (i - 1) % mod
    
n = int(input())

print(dp[n])

### 커스텀 기준 정렬

In [ ]:
import functools

def compare(i, j):
    ua, ub = u
    va, vb = v
    
    r1 = va * ub
    r2 = ua * vb
    
    if ua == 0:
        return 1
    elif va == 0:
        return -1
    elif ub == vb == 0:
        if ua > va:
            return 1
        else:
            return -1
    else:
        return r1 - r2
    
li.sort(key=functools.cmp_to_key(compare))

### 비트연산 특징

a+b=(a or b)+(a and  b)
기타: https://codeforces.com/blog/entry/94470